In [1]:
import matplotlib.pyplot as plt
import numpy as np
import imageio
import glob
import sys
import tensorflow as tf
import time

In [2]:
def get_img_array(path):                                                            
    """                                                                             
    Given path of image, returns it's numpy array                                   
    """                                                                             
    return imageio.imread(path)                                                  
                                                                                    
def get_files(folder):                                                              
    """                                                                             
    Given path to folder, returns list of files in it                               
    """                                                                             
    filenames = [file for file in glob.glob(folder+'*/*')]                          
    filenames.sort()                                                                
    return filenames                                                                
                                                                                    
def get_label(filepath, label2id):                                                  
    """                                                                             
    Files are assumed to be labeled as: /path/to/file/999_frog.png                  
    Returns label for a filepath                                                    
    """                                                                             
    tokens = filepath.split('/')                                                    
    label = tokens[-1].split('_')[1][:-4]                                           
    if label in label2id:                                                           
        return label2id[label]                                                      
    else:                                                                           
        sys.exit("Invalid label: " + label)

In [3]:
def get_labels(folder, label2id):                                                
    """                                                                          
    Returns vector of labels extracted from filenames of all files in folder     
    :param folder: path to data folder                                           
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """                                                                          
    files = get_files(folder)                                                    
    y = []                                                                       
    for f in files:                                                              
        y.append(get_label(f,label2id))                                          
    return np.array(y)                                                                                                                                                                                   
                                                                                 
def get_label_mapping(label_file):                                               
    """                                                                          
    Returns mappings of label to index and index to label                        
    The input file has list of labels, each on a separate line.                  
    """                                                                          
    with open(label_file, 'r') as f:                                             
        id2label = f.readlines()                                                 
        id2label = [l.strip() for l in id2label]                                 
    label2id = {}                                                                
    count = 0                                                                    
    for label in id2label:                                                       
        label2id[label] = count                                                  
        count += 1                                                               
    return id2label, label2id 

def get_images(folder):                                                          
    """                                                                          
    returns numpy array of all samples in folder                                 
    each column is a sample resized to 30x30 and flattened                       
    """                                                                          
    files = get_files(folder)                                                    
    images = []                                                                  
    count = 0                                                                    

    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr / 255.0
        images.append(img_arr)
    X = np.stack(images)

    return X

def get_train_data(data_root_path):                                              
    """                                                                          
    Return X and y                                                               
    """                                                                          
    train_data_path = data_root_path + 'train'                                   
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')          
    X = get_images(train_data_path)                                              
    y = get_labels(train_data_path, label2id)                                    
    return X, y                                                                  
                                                                                 
def save_predictions(filename, y):                                               
    """                                                                          
    Dumps y into .npy file                                                       
    """                                                                          
    np.save(filename, y)

In [4]:
data_root_path = 'cifar10-hw2/'
train_x, train_y = get_train_data(data_root_path) # this may take a few minutes
test_x = get_images(data_root_path + 'test')
print('Data loading done')

Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
""" Use to reset Tensorboard graph visualizations across runs """
def reset_graph():
    tf.reset_default_graph()
    tf.set_random_seed(42)
    np.random.seed(42)

""" 
    - Creates dir "tf_logs" in working direction to store Tensorboard events
    - Runs will appear w/in tf_logs directory as 'run-{timestamp}' to be used by TB
"""
reset_graph()

In [15]:
class CIFAR10_CNN(object):
    def __init__(self, X_data, Y_data, percentage_test=.1, random_seed=42):
        reset_graph()
        
        # model from memmory
        tf.set_random_seed(42)
        np.random.seed(42)
        self.perc_test = percentage_test
        self.num_examples = X_data.shape[0]
        cutoff = int(self.num_examples * self.perc_test)
        
        indices = np.arange(self.num_examples)
        np.random.shuffle(indices)
        test_indices = indices[:cutoff]
        train_indices = indices[cutoff:]

        # select training sets w/ holdout
        self.train_x = X_data[train_indices, :, :, :]
        self.train_y = Y_data[train_indices]

        # select testing sets w/ holdout
        self.test_x = train_x[test_indices, :, :, :]
        self.test_y = train_y[test_indices]
    
    def build_graph(self, convolutional_dimensions=[64, 128], dense_layer_dimensions=[1024, 128]):

        with tf.name_scope("input"):
            last_layer = tf.placeholder(tf.float32, [None, 32, 32, 3], name="x_input")
            y = tf.placeholder(tf.int64, [None], name="y_input")
            global_step = tf.placeholder(tf.int64, [], name="global_step")
            
        # Normalize input on input data
        with tf.name_scope("input_normalization"):
            last_layer = tf.layers.batch_normalization(last_layer)

        # add convolutional layers
        for idx, output_dim in enumerate(convolutional_dimensions):
            with tf.name_scope("convolutions"):
                last_layer = self.get_conv_layer(last_layer, output_dim, [3, 3], "SAME", tf.nn.relu)
            with tf.name_scope("max_pooling"):
                last_layer = tf.layers.max_pooling2d(last_layer, pool_size=[3, 3], strides=2, padding="SAME")
            #with tf.name_scope("normalization"):
                #last_layer = tf.layers.batch_normalization(last_layer)
        
        # flatten the last convolution
        last_layer = tf.contrib.layers.flatten(last_layer)

        # add dense layers
        for idx, output_dim in enumerate(dense_layer_dimensions):
            with tf.name_scope("dense"):
                last_layer = tf.layers.dense(inputs=last_layer, units=output_dim, activation=tf.nn.relu)

        # last layer is always dimension (?, 10)
        logits = tf.layers.dense(inputs=last_layer, units=10, name="logits")
        
    def define_optimization(self, learning_rate=0.01):
        default_graph = tf.get_default_graph()
        y = default_graph.get_tensor_by_name("input/y_input:0")
        global_step = default_graph.get_tensor_by_name("input/global_step:0")
        logits = default_graph.get_tensor_by_name("logits/BiasAdd:0")
        
        with tf.name_scope("cross_entropy"):
                xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, 10), logits=logits)
                with tf.name_scope("loss"):
                    loss = tf.reduce_mean(xentropy, name="loss")
        
        with tf.name_scope("softmax"):
            # Softmax layer
            softmax = tf.nn.softmax(logits)
            # Predict train
            y_hat = tf.argmax(softmax, axis=1, name="y_hat")
            
        with tf.name_scope('train'):
            optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(xentropy, name="training_op")
 
        with tf.name_scope("accuracy"):
            # Train Evaluation
            train_correct = tf.equal(y, y_hat)
            tf.reduce_mean(tf.cast(train_correct, tf.float64), name="acc")


    def train(self, sess, train_x=None, train_y=None, n_epochs=1, batch_size=100):
        
        # New***
        init = tf.global_variables_initializer() 
        
        # so we can load the graph into memory and train with data that we input
        # or data that we've saved
        if train_x == None:
            train_x = self.train_x
        if train_y == None:
            train_y = self.train_y

        # get default graph
        default_graph = tf.get_default_graph()
        
        # Get individual graph tensors
        X = default_graph.get_tensor_by_name("input/x_input:0")
        Y = default_graph.get_tensor_by_name("input/y_input:0")
        global_step = default_graph.get_tensor_by_name("input/global_step:0")
        
        # Get specific operations in graph
        training_op = default_graph.get_operation_by_name("train/training_op")
        accuracy = default_graph.get_tensor_by_name("accuracy/acc:0")
        loss = default_graph.get_tensor_by_name("cross_entropy/loss/loss:0")
        
        # Create logdir
        now = time.strftime("%Y%m%d%H%M%S")
        root_logdir = "tf_logs"
        logdir = "{}/run-{}/".format(root_logdir, now)
        logdir = f"{root_logdir}/run"
        
        # Create Tensorboard file_writers
        summary_writer = tf.summary.FileWriter(logdir, default_graph)
        
        # Create Tensorboard Summaries 
        acc_train_summary = tf.summary.scalar("train_accuracy", accuracy)
        acc_test_summary = tf.summary.scalar("test_accuracy", accuracy)
        loss_summary = tf.summary.scalar('train_loss', loss)
        
        # Create Tensorboard histograms
        conv_histograms = []
        for i in tf.global_variables():
            if i.name[:6] == "conv2d":
                # need to switch colon with underscore because colons aren't allowed in summary names
                conv_histograms.append(tf.summary.histogram(i.name.replace(":", "_"), i))
        
        # Wrapper tells Tensorboard to assume everything is within same session
        sess.run(init)

        step = 0
        for epoch in range(1, n_epochs + 1):
            n_iters = int((self.num_examples * (1 - self.perc_test)) / batch_size)
            print_interval = 100
            best_test = 0.

            print(f"\nStarting epoch {epoch}, running for {n_iters} iterations")
            print(f"Printing evaluation metrics every {print_interval} iterations\n")

            for iteration in range(1, n_iters):
                step += 1
                indices = np.random.choice(int(self.num_examples * (1 - self.perc_test)), batch_size)
                X_batch = self.train_x[indices]
                y_batch = self.train_y[indices]
                sess.run(training_op, feed_dict={X: X_batch,
                                                 Y: y_batch,
                                                 global_step: step})

                if iteration % print_interval == 0:

                    acc_train, train_summary = sess.run([accuracy, tf.summary.merge([loss_summary, acc_train_summary])],
                                                        {X: X_batch, Y: y_batch})

                    acc_test, test_summary = sess.run([accuracy, acc_test_summary], {X: self.test_x, Y: self.test_y})

                    # Add batch train loss & accuracy to Tensorboard
                    merged_histograms = tf.summary.merge([i.eval(session=sess) for i in conv_histograms])
                    summary_writer.add_summary(tf.summary.merge([train_summary, test_summary, merged_histograms]).eval(session=sess), step)
                    summary_writer.flush()

                    # Print out batch evaluation metrics
                    print("Iteration: {} train acc: {:.4f} test acc: {:.4f}".format(iteration, 
                                                                                    acc_train, 
                                                                                    acc_test))
        # Ensure file_writers closed
        summary_writer.close() 
        
    def get_conv_layer(self, input_layer, filter_num, ksize, padding, activation):

        conv_layer = tf.layers.conv2d(inputs = input_layer,
                                     filters = filter_num,
                                     kernel_size = ksize,
                                     padding = padding,
                                     activation=activation)
        return conv_layer

In [16]:
cnn = CIFAR10_CNN(train_x, train_y)
cnn.build_graph(convolutional_dimensions=[96,192], dense_layer_dimensions=[1024])
cnn.define_optimization(learning_rate=0.01)
sess = tf.Session()
cnn.train(sess, train_x=None, train_y=None, n_epochs=20, batch_size=100)


Starting epoch 1, running for 450 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.3900 test acc: 0.2822
Iteration: 200 train acc: 0.5300 test acc: 0.4666
Iteration: 300 train acc: 0.6600 test acc: 0.5406
Iteration: 400 train acc: 0.7100 test acc: 0.5760

Starting epoch 2, running for 450 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.7100 test acc: 0.6296
Iteration: 200 train acc: 0.7900 test acc: 0.6514
Iteration: 300 train acc: 0.6800 test acc: 0.6598
Iteration: 400 train acc: 0.7600 test acc: 0.6636

Starting epoch 3, running for 450 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 train acc: 0.7900 test acc: 0.6774
Iteration: 200 train acc: 0.8600 test acc: 0.6898
Iteration: 300 train acc: 0.7500 test acc: 0.7002
Iteration: 400 train acc: 0.8500 test acc: 0.6918

Starting epoch 4, running for 450 iterations
Printing evaluation metrics every 100 iterations

Iteration: 100 t

In [17]:
# After training, run this cell to generate the predictions file ans1-uni.npy
predictions_filename = "ans1-uni.npy"
g = tf.get_default_graph()
logits = g.get_operation_by_name("logits/BiasAdd").outputs[0]
X = g.get_tensor_by_name("input/x_input:0")

# sess should still exist from training in the previous cell
y_unnormalized_prediction = sess.run(logits, {X: test_x}).T
with open(predictions_filename, "wb+") as f:
    np.save(f, y_unnormalized_prediction)

# confirm saved values are in the right dimension (10, N)
with open(predictions_filename, "rb") as f:
    v = np.load(f)
    print(v.shape)

(10, 10000)
